# Final Project

## Environment Setup

In [2]:
import os
from PIL import Image
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoFeatureExtractor, AutoModel

## Custom DogVsAi Dataset Class

In [3]:
class DogVsAiDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_dir = os.path.join(root_dir, "Images")
        self.label_dir = os.path.join(root_dir, "Labels")
        self.transform = transform
        self.image_names = [f.split('.')[0] for f in os.listdir(self.image_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        name = self.image_names[idx]
        image_path = os.path.join(self.image_dir, name + '.jpg')
        label_path = os.path.join(self.label_dir, name + '.txt')

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        with open(label_path, 'r') as f:
            label = int(f.read().strip())

        return image, label

## ViT Feature Extractor Setup: Image Transformations and DINO Model Initialization

In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dino_model_name = "facebook/dino-vits16"
feature_extractor = AutoFeatureExtractor.from_pretrained(dino_model_name)
dino_model = AutoModel.from_pretrained(dino_model_name).to(device)
dino_model.eval()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vits16 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=384, out_features=1536, bias=True)
          (intermediate_act_fn): GELUAct

## Feature Extraction Pipeline and Dataset Processing

In [5]:
# ======================
# Feature Engineering
# ======================
def extract_and_save_features(dataloader, name_prefix):
    all_features = []
    all_labels = []

    for images, labels in tqdm(dataloader, desc=f"Extracting {name_prefix}"):
        images = [transforms.ToPILImage()(img) for img in images]
        inputs = feature_extractor(images=images, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            outputs = dino_model(**inputs)
            features = outputs.last_hidden_state.mean(dim=1)

        all_features.append(features.cpu())
        all_labels.append(labels)

    all_features = torch.cat(all_features)
    all_labels = torch.cat(all_labels)

    save_path = f"features_{name_prefix}.pt"
    torch.save((all_features, all_labels), save_path)
    print(f"Saved: {save_path} | Shape: {all_features.shape}")

# ======================
# Process dataset
# ======================
def process_dataset(base_path, suffix):
    print(f"\n=== Processing dataset: {suffix.upper()} ===")
    train_dataset = DogVsAiDataset(os.path.join(base_path, "Train"), transform=transform)
    val_dataset   = DogVsAiDataset(os.path.join(base_path, "Valid"), transform=transform)
    test_dataset  = DogVsAiDataset(os.path.join(base_path, "Test"), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)

    print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

    extract_and_save_features(train_loader, f"train_{suffix}")
    extract_and_save_features(val_loader, f"val_{suffix}")
    extract_and_save_features(test_loader, f"test_{suffix}")

# ======================
# Get features
# ======================
process_dataset("archive/Dogs Vs AiDogs", "full")
# process_dataset("archive/Dogs Vs AiDogs_CUTTED", "cutted")


=== Processing dataset: FULL ===
Train: 18605 | Val: 5317 | Test: 2658


Extracting train_full:   0%|          | 0/1163 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)
Extracting train_full: 100%|██████████| 1163/1163 [05:36<00:00,  3.46it/s]


Saved: features_train_full.pt | Shape: torch.Size([18605, 384])


Extracting val_full: 100%|██████████| 333/333 [01:40<00:00,  3.30it/s]


Saved: features_val_full.pt | Shape: torch.Size([5317, 384])


Extracting test_full: 100%|██████████| 167/167 [00:51<00:00,  3.27it/s]

Saved: features_test_full.pt | Shape: torch.Size([2658, 384])


In [6]:
features, labels = torch.load("features_train_full.pt")
print(features.shape)

torch.Size([18605, 384])


# Simple CNN Architecture with Dropout

In [7]:
import torch.nn as nn
import torchvision.models as models

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(384, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

def load_model(model_type="simple", num_classes=2, dropout_rate=0.5):
    return SimpleCNN(num_classes, dropout_rate)